# Inicialización

## Carga de librerias

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

## Configuracion de entorno

In [208]:
pd.set_option('display.max_rows', 15)

In [209]:
pd.options.display.float_format = '{:,.0f}'.format

# Carga y limpieza de datos

## Carga de base de datos

In [210]:
pelis = pd.read_csv("TMDB_movie_dataset_v11.csv")
pelis.head(3)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",84,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,9,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,131,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."


In [211]:
pelis.shape

(1074114, 24)

In [212]:
pelis.dtypes

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
                         ...   
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
Length: 24, dtype: object

In [213]:
pelis.isnull().sum()

id                           0
title                       12
vote_average                 0
vote_count                   0
status                       0
                         ...  
genres                  413773
production_companies    577029
production_countries    457742
spoken_languages        441454
keywords                773703
Length: 24, dtype: int64

In [214]:
(pelis == 0).sum()

id                           0
title                        0
vote_average            725553
vote_count              725325
status                       0
                         ...  
genres                       0
production_companies         0
production_countries         0
spoken_languages             0
keywords                     0
Length: 24, dtype: int64

## Eliminación de columnas y filas no relevantes 

In [215]:
# se eliminan columnas que no son de intrés para el análisis
cols_drop = [
    'backdrop_path',
    'homepage',
    'imdb_id',
    'original_title',
    'overview',
    'popularity',
    'poster_path',
    'tagline',
    'production_companies',
    'spoken_languages',
    'overview', 
    'keywords'
]

pelis_clean = pelis.drop(columns = cols_drop)
pelis_clean.head(2)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries
0,27205,Inception,8,34495,Released,2010-07-15,825532764,148,False,160000000,en,"Action, Science Fiction, Adventure","United Kingdom, United States of America"
1,157336,Interstellar,8,32571,Released,2014-11-05,701729206,169,False,165000000,en,"Adventure, Drama, Science Fiction","United Kingdom, United States of America"


In [216]:
#se consideran cortometrajes (runtmie>60min) hasta 4 horas (runtime,240min)
pelis_clean = pelis_clean[(pelis_clean.runtime > 60) & (pelis_clean.runtime <240)]

In [217]:
# se eliminan peliculas en produccion
pelis_clean = pelis_clean[pelis_clean['status'] != 'In Production']

In [218]:
pelis_clean.shape

(420661, 13)

In [219]:
pelis_clean.isnull().sum()

id                           0
title                        1
vote_average                 0
vote_count                   0
status                       0
release_date             20064
revenue                      0
runtime                      0
adult                        0
budget                       0
original_language            0
genres                  119638
production_countries    111715
dtype: int64

In [220]:
(pelis_clean == 0).sum()

id                           0
title                        0
vote_average            201288
vote_count              201178
status                       0
release_date                 0
revenue                 404282
runtime                      0
adult                   351971
budget                  394833
original_language            0
genres                       0
production_countries         0
dtype: int64

## Tratamiento de nulos y ceros 

### Eliminación de datos que no pueden ser completados 

In [221]:
# las pelis con nulos en el titulo les falta información en otras columnas, se eliminan
pelis_clean = pelis_clean.dropna(subset=['title'])

In [222]:
#se eliminan pelis con 'release_date' nulo, se da formato a fecha y se agrega columna 'year' tipo int
pelis_clean = pelis_clean.dropna(subset=['release_date'])
pelis_clean.release_date = pd.to_datetime(pelis_clean.release_date, errors='coerce')
pelis_clean['year'] = pd.to_datetime(pelis_clean.release_date.dt.year, format='%Y')

In [223]:
# se eliminan filas con valores = 0 en las columnas revenue y budget
pelis_clean = pelis_clean[(pelis_clean['revenue'] != 0) & (pelis_clean['budget'] != 0)]

In [224]:
#revisar valores duplicados
tit_dupl = pelis_clean.title.value_counts()#[tit_dupl_cant>1]
tit_dupl[tit_dupl>1].size

254

In [225]:
tit_dupl[tit_dupl>1]

title
Little Women       3
King Kong          3
Pinocchio          3
A Star Is Born     3
Black Christmas    3
                  ..
The Blob           2
The In-Laws        2
Vice               2
The Hunted         2
Dracula            2
Name: count, Length: 254, dtype: int64

In [226]:
pelis_clean[pelis_clean['title'] == 'A Star Is Born']

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries,year
225,332562,A Star Is Born,8,11079,Released,2018-10-03,433888866,136,False,36000000,en,"Music, Drama, Romance",United States of America,2018-01-01
9725,3111,A Star Is Born,7,266,Released,1954-10-01,4335968,176,False,5019770,en,"Drama, Music, Romance",United States of America,1954-01-01
11754,19610,A Star Is Born,6,200,Released,1976-12-17,80000000,139,False,6000000,en,"Drama, Music, Romance",United States of America,1976-01-01


In [227]:
pelis_clean.id.duplicated().sum()

0

*Nota:*  
Se encuentran películas con el mismo nombre, pero el id y el resto de las características son diferentes. No es necesario elimianar estos duplicados

In [228]:
pelis_clean.shape

(10221, 14)

### Completación de datos 

In [229]:
pelis_clean.isnull().sum()

id                        0
title                     0
vote_average              0
vote_count                0
status                    0
release_date              0
revenue                   0
runtime                   0
adult                     0
budget                    0
original_language         0
genres                   71
production_countries    305
year                      0
dtype: int64

In [230]:
(pelis_clean == 0).sum()

id                          0
title                       0
vote_average              393
vote_count                391
status                      0
release_date                0
revenue                     0
runtime                     0
adult                   10185
budget                      0
original_language           0
genres                      0
production_countries        0
year                        0
dtype: int64

*Nota*  
Se establecen los siguientes criterios para complatar faltantes y ceros:
- 'production_countries': moda según 'original_language'
- 'genres': moda según década 
- 'vote_average': promedio segun década y 'genres'
- 'vote_count': promedio segun decada

#### Completar los na de 'production_countries' 

In [231]:
#Calcular la moda para cada 'original_language'
moda_por_idioma = pelis_clean.groupby('original_language')['production_countries'].\
    agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

#Rellenar los valores NaN en 'production_countries' usando la moda calculada
def rellenar_moda(row, moda_series):
    if pd.isna(row['production_countries']):
        return moda_series.get(row['original_language'], row['production_countries'])
    return row['production_countries']

pelis_clean['production_countries'] = pelis_clean.apply(rellenar_moda, moda_series=moda_por_idioma, axis=1)

In [232]:
#chequeo na en 'production_countries'
pelis_clean[pelis_clean.production_countries.isna()]

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries,year
272371,656944,Twice Upon a Time in the West,8,1,Released,2015-10-07,1045,97,False,610000,bg,"Comedy, Thriller, Drama",None,2015-01-01
293465,1024777,Bai Ivan 2,1,1,Released,2022-09-09,60000,95,False,10000,bg,"Comedy, Adventure",None,2022-01-01


In [233]:
pelis_clean[pelis_clean.original_language == 'bg']

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries,year
272371,656944,Twice Upon a Time in the West,8,1,Released,2015-10-07,1045,97,False,610000,bg,"Comedy, Thriller, Drama",None,2015-01-01
293465,1024777,Bai Ivan 2,1,1,Released,2022-09-09,60000,95,False,10000,bg,"Comedy, Adventure",None,2022-01-01


*Nota*  
Quedan dos elementos sin rellenar. Se comprueba que son las unicas películas con 'original_language' 'bg' (búlgaro). Completamos el 'production_countries' de estos elementos con 'Bulgaria'

In [234]:
#Completamos el 'production_countries' de estos elementos con 'Bulgaria'
pelis_clean.loc[(pelis_clean['original_language'] == 'bg') & (pelis_clean['production_countries'].isna()), 'production_countries'] = 'Bulgaria'

#### Completar los na de 'genres' 

In [235]:
# Crear una nueva columna 'decade' que contenga la década
pelis_clean['decade'] = (pelis_clean['year'].dt.year // 10) * 10

#busco la moda de genero por década
moda_por_decada = pelis_clean.groupby('decade')['genres'].agg(lambda x: x.mode().iloc[0])

#completo los géneros faltates con la moda por decada
pelis_clean['genres'] = pelis_clean.apply(lambda row: moda_por_decada[row['decade']] if pd.isna(row['genres']) else row['genres'], axis=1)

#### Completar los ceros de 'vote_average' 

In [236]:
# Crear una máscara para los valores de 'vote_average' que son 0
mask = pelis_clean['vote_average'] == 0

# Calcular el promedio de 'vote_average' agrupado por 'decade' y 'genres'
grouped_means = pelis_clean[~mask].groupby(['decade', 'genres'])['vote_average'].mean()

# Crear una función para reemplazar los valores 0 con el promedio correspondiente
def fill_zeros(row):
    if row['vote_average'] == 0:
        return grouped_means.get((row['decade'], row['genres']), row['vote_average'])
    else:
        return row['vote_average']

# Aplicar la función al DataFrame
pelis_clean['vote_average'] = pelis_clean.apply(fill_zeros, axis=1)

In [237]:
pelis_clean[pelis_clean.genres == 'Romance, Family, TV Movie, Comedy']

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries,year,decade
378984,1108927,Dhakad Chhora,0,0,Released,2004-10-08,500000,145,False,100000,hi,"Romance, Family, TV Movie, Comedy",India,2004-01-01,2000


In [238]:
#chequeo ceros en 'production_countries'
pelis_clean[pelis_clean.vote_average == 0].head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,genres,production_countries,year,decade
349915,1033775,Hood Fetish: The Era,0,0,Released,2022-10-18,100,77,False,70,en,"Drama, Crime",United States of America,2022-01-01,2020
378984,1108927,Dhakad Chhora,0,0,Released,2004-10-08,500000,145,False,100000,hi,"Romance, Family, TV Movie, Comedy",India,2004-01-01,2000
379422,84132,Kanske En Gentleman,0,0,Released,1950-12-26,60,91,False,50,sv,"Drama, Comedy",Sweden,1950-01-01,1950
393109,1127007,Dok,0,0,Released,2022-04-06,3000000,92,False,1000000,tl,"Drama, Family",Philippines,2022-01-01,2020
393821,1124489,One Minnesota,0,0,Released,2020-03-20,9000,70,False,10000,en,"Horror, Romance, Mystery, History, TV Movie","Slovenia, United States of America",2020-01-01,2020


*Nota*  
Se encuentran combinaciones de 'decade' y 'genres' que no tienen elementos, por lo que no hay promedios.
En esos caso se rellenan con promedio según 'genres' primero, 'decade' si es que no existe.

In [239]:
# Calcular el promedio de de 'vote_average agrupado solo por 'genres'
grouped_means_genres = pelis_clean[~mask].groupby('genres')['vote_average'].mean()

# Crear una función para reemplazar los valores 0 con el promedio correspondiente al género
def fill_zeros(row):
    if row['vote_average'] == 0:
        return grouped_means_genres.get(row['genres'], row['vote_average'])
    else:
        return row['vote_average']

# Aplicar la función al DataFrame
pelis_clean['vote_average'] = pelis_clean.apply(fill_zeros, axis=1)

In [240]:
# Calcular el promedio de 'vote_average' agrupado solo por 'decade'
grouped_means_decade = pelis_clean[~mask].groupby('decade')['vote_average'].mean()

# Crear una función para reemplazar los valores 0 con el promedio correspondiente de la década
def fill_zeros(row):
    if row['vote_average'] == 0:
        # Obtener el promedio de la década para la fila
        return grouped_means_decade.get(row['decade'], row['vote_average'])
    else:
        return row['vote_average']

# Aplicar la función al DataFrame
pelis_clean['vote_average'] = pelis_clean.apply(fill_zeros, axis=1)

#### Completar los ceros de 'vote_count' 

In [241]:
media_vote_count_por_decada = pelis_clean.groupby('decade')['vote_count'].mean().round().astype(int)
pelis_clean.loc[pelis_clean['vote_count'] == 0, 'vote_count'] = pelis_clean.loc[pelis_clean['vote_count'] == 0, 'decade'].map(media_vote_count_por_decada)

In [242]:
pelis_clean.isnull().sum()

id                      0
title                   0
vote_average            0
vote_count              0
status                  0
release_date            0
revenue                 0
runtime                 0
adult                   0
budget                  0
original_language       0
genres                  0
production_countries    0
year                    0
decade                  0
dtype: int64

In [243]:
(pelis_clean == 0).sum()

id                          0
title                       0
vote_average                0
vote_count                  0
status                      0
release_date                0
revenue                     0
runtime                     0
adult                   10185
budget                      0
original_language           0
genres                      0
production_countries        0
year                        0
decade                      0
dtype: int64

## Convertir a precios constantes

Agregamos columnas  'budget_adjus' y 'revenue_adjus' ajsutando las originales por IPC con base 2024, tomando como promedio de inflacion anual 3,2%

In [244]:
# Definir el aumento anual promedio del IPC
ipc_increase_rate = 0.032  # 3.2% en formato decimal

# Obtener el año actual
current_year = datetime.now().year

# Asegurarte de que 'release_date' sea un año entero
def extraer_anio(fecha):
    try:
        return pd.to_datetime(fecha).year
    except:
        return None

# Función para ajustar el precio por IPC
def ajustar_precio(precio, release_date):
    if pd.isna(precio) or pd.isna(release_date):
        return None
    release_year = extraer_anio(release_date)
    if release_year is None or release_year > current_year:
        return None  # Si el año de lanzamiento es futuro o no se puede extraer, no tiene sentido ajustar
    num_years = current_year - release_year
    # Ajuste del presupuesto usando la fórmula de interés compuesto
    precio_ajustado = precio * ((1 + ipc_increase_rate) ** num_years)
    return precio_ajustado

# Crear la nueva columna 'budget_adjust'
pelis_clean['budget_adjust'] = pelis_clean.apply(
    lambda row: ajustar_precio(row['budget'], row['release_date']), axis=1
)

# Crear la nueva columna 'revenue_adjust'
pelis_clean['revenue_adjust'] = pelis_clean.apply(
    lambda row: ajustar_precio(row['revenue'], row['release_date']), axis=1
)

# Mostrar las primeras filas para verificar
print(pelis_clean[[ 'release_date', 'revenue', 'revenue_adjust', 'budget', 'budget_adjust']].head())

  release_date     revenue  revenue_adjust     budget  budget_adjust
0   2010-07-15   825532764   1,283,069,190  160000000    248,677,072
1   2014-11-05   701729206     961,538,161  165000000    226,089,773
2   2008-07-16  1004558444   1,662,839,626  185000000    306,229,401
3   2009-12-15  2923706026   4,689,528,313  237000000    380,140,206
4   2012-04-25  1518815515   2,216,467,626  220000000    321,054,712


In [245]:
#elimino las columnas originales
pelis_clean = pelis_clean.drop(['revenue', 'budget'], axis=1)

In [246]:
pelis_clean.head(2)

,id,title,vote_average,vote_count,status,release_date,runtime,adult,original_language,genres,production_countries,year,decade,budget_adjust,revenue_adjust
0,27205,Inception,8,34495,Released,2010-07-15,148,False,en,"Action, Science Fiction, Adventure","United Kingdom, United States of America",2010-01-01,2010,"248,677,072","1,283,069,190"
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,en,"Adventure, Drama, Science Fiction","United Kingdom, United States of America",2014-01-01,2010,"226,089,773","961,538,161"


## Agrupar por géneros

In [247]:
# Asegurarse de que todos los valores sean cadenas de texto
pelis_clean['genres'] = pelis_clean['genres'].astype(str)

# Dividir las cadenas en listas de valores
generos = pelis_clean['genres'].str.split(', ', expand=False)

# Aplanar la lista de listas en una sola lista, excluyendo 'nan'
generos_aplanados = [valor for sublist in generos for valor in sublist if valor != 'nan']

# Convertir la lista a un conjunto para eliminar duplicados y luego a una lista
generos_unicos = list(set(generos_aplanados))

# Ordenar los valores (opcional)
generos_unicos.sort()

generos_unicos

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [248]:
diccionario_generos = {
    "Accion/Aventura": [
        "Action",
        "Adventure",
        "Science Fiction",
        "Western",
        "War"
    ],
    "Drama/Romance": [
        "Drama",
        "Romance"
    ],
    "Todos los Publicos": [
        "Comedy",
        "Family",
        "Animation",
        "Fantasy",
        "Music"
    ],
    "Terror/Misterio": [
        "Horror",
        "Mystery",
        "Crime",
        "Thriller"
    ],
    "Documental/TV_Movie": [
        "Documentary",
        "TV Movie",
        "History"
    ]
}

# Crear un diccionario inverso para mapeo
diccionario_mapeo = {v: k for k, valores in diccionario_generos.items() for v in valores}

# Función para mapear los valores usando el diccionario
def mapear_valores(cadena, diccionario):
    valores = cadena.split(', ')
    valores_mapeados = {diccionario.get(valor, valor) for valor in valores}
    return ', '.join(valores_mapeados)

# Aplicar la función al DataFrame
pelis_clean['genres'] = pelis_clean['genres'].apply(lambda x: mapear_valores(x, diccionario_mapeo))

# Dividir las cadenas en listas de valores
generos_listas = pelis_clean['genres'].str.split(', ', expand=False)

# Explode y convertir en dummies
generos_dummies = pd.get_dummies(generos_listas.explode()).groupby(level=0).max()

# Unir el DataFrame original con el DataFrame de dummies
pelis_genres = pd.concat([pelis_clean, generos_dummies], axis=1)

# Eliminar la columna original si ya no es necesaria
pelis_genres = pelis_genres.drop(columns=['title','genres', 'decade', 'vote_average', 'vote_count', 'status', 'release_date',
       'runtime', 'adult', 'original_language',
       'production_countries', 'year', 'budget_adjust', 'revenue_adjust'])

pelis_genres

,id,Accion/Aventura,Documental/TV_Movie,Drama/Romance,Terror/Misterio,Todos los Publicos
0,27205,True,False,False,False,False
1,157336,True,False,True,False,False
2,155,True,False,True,True,False
3,19995,True,False,False,False,True
4,24428,True,False,False,False,False
...,...,...,...,...,...,...
1060310,290067,False,False,True,False,True
1063844,756895,False,False,True,False,False
1066407,761273,False,False,False,False,True
1069361,750182,True,False,False,True,False


In [249]:
pelis_clean.head()

,id,title,vote_average,vote_count,status,release_date,runtime,adult,original_language,genres,production_countries,year,decade,budget_adjust,revenue_adjust
0,27205,Inception,8,34495,Released,2010-07-15,148,False,en,Accion/Aventura,"United Kingdom, United States of America",2010-01-01,2010,"248,677,072","1,283,069,190"
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,en,"Accion/Aventura, Drama/Romance","United Kingdom, United States of America",2014-01-01,2010,"226,089,773","961,538,161"
2,155,The Dark Knight,9,30619,Released,2008-07-16,152,False,en,"Terror/Misterio, Accion/Aventura, Drama/Romance","United Kingdom, United States of America",2008-01-01,2000,"306,229,401","1,662,839,626"
3,19995,Avatar,8,29815,Released,2009-12-15,162,False,en,"Todos los Publicos, Accion/Aventura","United States of America, United Kingdom",2009-01-01,2000,"380,140,206","4,689,528,313"
4,24428,The Avengers,8,29166,Released,2012-04-25,143,False,en,Accion/Aventura,United States of America,2012-01-01,2010,"321,054,712","2,216,467,626"


*Nota*  
El código anterrior devuelve:
- pelis_clean con los generos agrupados segun el diccionario
- pelis_genres con el id y los generos en formato dummies

## Agrupar por paises y continentes

In [250]:
# Asegurarse de que todos los valores sean cadenas de texto
pelis_clean['production_countries'] = pelis_clean['production_countries'].astype(str)

# Dividir las cadenas en listas de valores
countries = pelis_clean['production_countries'].str.split(', ', expand=False).reset_index()
countries

# Expandir las listas en filas individuales
countries_exploded = countries.explode('production_countries')
countries_exploded

# Agrupar por países y contar
conteo_paises = countries_exploded['production_countries'].value_counts().reset_index()
conteo_paises.columns = ['Country', 'Count']

# Mostrar el resultado
conteo_paises.head(15)

,Country,Count
0,United States of America,7238
1,United Kingdom,1153
2,France,780
3,India,731
4,Germany,525
5,Canada,466
6,Japan,259
7,Spain,234
8,Italy,230
9,Russia,224


In [251]:
conteo_paises.Country.unique()

array(['United States of America', 'United Kingdom', 'France', 'India',
       'Germany', 'Canada', 'Japan', 'Spain', 'Italy', 'Russia',
       'Australia', 'China', 'Belgium', 'Hong Kong', 'South Korea',
       'Iran', 'Sweden', 'Ireland', 'Mexico', 'Netherlands', 'Turkey',
       'Denmark', 'Argentina', 'Brazil', 'Norway', 'Czech Republic',
       'Switzerland', 'Hungary', 'Finland', 'New Zealand', 'Thailand',
       'Luxembourg', 'Bulgaria', 'Egypt', 'Romania', 'Pakistan',
       'South Africa', 'Taiwan', 'Poland', 'Austria',
       'United Arab Emirates', 'Philippines', 'Singapore', 'Malaysia',
       'Ukraine', 'Israel', 'Morocco', 'Greece', 'Peru', 'Vietnam',
       'Indonesia', 'Portugal', 'Iceland', 'Chile', 'Malta', 'Bangladesh',
       'Colombia', 'Serbia', 'Uruguay', 'Slovakia', 'Soviet Union',
       'Lebanon', 'Nepal', 'Puerto Rico', 'Qatar', 'Estonia', 'Venezuela',
       'Aruba', 'Maldives', 'Yugoslavia', 'Slovenia', 'Georgia',
       'Cambodia', 'Paraguay', 'Nigeria', '

In [252]:
import pandas as pd

# Diccionario de países agrupados por continente
diccionario_countries = {
    "Estados Unidos": ["United States of America"],
    "Reino Unido": ["United Kingdom"],
    "Francia": ["France"],
    "India": ["India"],
    "Alemania": ["Germany"],
    "Canada": ["Canada"],
    "Japon": ["Japan"],
    "España": ["Spain"],
    "Italia": ["Italy"],
    "Rusia": ["Russia"],
    'Africa': ['Algeria', 'American Samoa', 'Angola', 'Benin', 'Botswana', 'Burkina Faso',
               'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic',
               'Chad', 'Comoros', 'Congo',"Cote D'Ivoire", 'Djibouti', 'Egypt',
               'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon',
               'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast',
               'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi',
               'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia',
               'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal',
               'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa',
               'South Sudan', 'Sudan', 'Togo', 'Tunisia', 'Uganda', 'United Republic of Tanzania',
               'Zambia', 'Zimbabwe'],
    'Asia': ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh',
             'Bhutan', 'Brunei', 'Cambodia', 'China', 'Cyprus', 'Georgia', 'Hong Kong','Indonesia',
             'Iran', 'Iraq', 'Israel', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan',
             'Laos', 'Lebanon', 'Libyan Arab Jamahiriya', 'Macao', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal',
             'Oman', 'Pakistan', 'Palestinian Territory', 'Philippines', 'Qatar',
             'Saudi Arabia', 'Singapore', 'South Korea', 'Soviet Union', 'Sri Lanka', 'Syria',
             'Taiwan', 'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey',
             'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam'],
    'Europe': ['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium',
               'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
               'Czech Republic', 'Denmark', 'Estonia', 'Finland',
               'Greece','Guadaloupe', 'Hungary', 'Iceland', 'Ireland', 'Kosovo',
               'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macedonia', 'Malta', 'Moldova',
               'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway',
               'Poland', 'Portugal', 'Romania', 'San Marino', 'Serbia', 'Slovakia',
               'Slovenia', 'Sweden', 'Switzerland', 'Ukraine', 'Yugoslavia', 'East Germany'],
    'Oceania': ['Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia',
                'Nauru', 'New Zealand', 'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands',
                'Tonga', 'Tuvalu', 'Vanuatu'],
    'América': ['Argentina', 'Aruba', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
                'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela',
                'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Costa Rica',
                'Cuba', 'Dominica', 'Dominican Republic', 'El Salvador', 'Grenada',
                'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua',
                'Panama', 'Puerto Rico', 'Saint Kitts and Nevis', 'Saint Lucia',
                'Saint Vincent and the Grenadines', 'Trinidad and Tobago']
}

# Crear un diccionario inverso para mapeo
diccionario_mapeo = {v: k for k, valores in diccionario_countries.items() for v in valores}

# Función para mapear los valores usando el diccionario
def mapear_valores(cadena, diccionario):
    valores = cadena.split(', ')
    valores_mapeados = {diccionario.get(valor, valor) for valor in valores}
    return ', '.join(valores_mapeados)

# Aplicar la función al DataFrame
pelis_clean['production_countries'] = pelis_clean['production_countries'].apply(lambda x: mapear_valores(x, diccionario_mapeo))

# Dividir las cadenas en listas de valores
countries_listas = pelis_clean['production_countries'].str.split(', ', expand=False)

# Explode y convertir en dummies
countries_dummies = pd.get_dummies(countries_listas.explode()).groupby(level=0).max()

# Unir el DataFrame original con el DataFrame de dummies
pelis_countries = pd.concat([pelis_clean, countries_dummies], axis=1)

# Eliminar la columna original si ya no es necesaria
pelis_countries = pelis_countries.drop(columns=['title','genres', 'decade', 'vote_average', 'vote_count', 'status', 'release_date',
       'runtime', 'adult', 'original_language',
       'production_countries', 'year', 'budget_adjust', 'revenue_adjust'])

# Mostrar el resultado
pelis_countries.head()

,id,Africa,Alemania,América,Asia,Canada,España,Estados Unidos,Europe,Francia,India,Italia,Japon,Oceania,Reino Unido,Rusia
0,27205,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
1,157336,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
2,155,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
3,19995,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
4,24428,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [253]:
pelis_clean.head()

,id,title,vote_average,vote_count,status,release_date,runtime,adult,original_language,genres,production_countries,year,decade,budget_adjust,revenue_adjust
0,27205,Inception,8,34495,Released,2010-07-15,148,False,en,Accion/Aventura,"Estados Unidos, Reino Unido",2010-01-01,2010,"248,677,072","1,283,069,190"
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,en,"Accion/Aventura, Drama/Romance","Estados Unidos, Reino Unido",2014-01-01,2010,"226,089,773","961,538,161"
2,155,The Dark Knight,9,30619,Released,2008-07-16,152,False,en,"Terror/Misterio, Accion/Aventura, Drama/Romance","Estados Unidos, Reino Unido",2008-01-01,2000,"306,229,401","1,662,839,626"
3,19995,Avatar,8,29815,Released,2009-12-15,162,False,en,"Todos los Publicos, Accion/Aventura","Estados Unidos, Reino Unido",2009-01-01,2000,"380,140,206","4,689,528,313"
4,24428,The Avengers,8,29166,Released,2012-04-25,143,False,en,Accion/Aventura,Estados Unidos,2012-01-01,2010,"321,054,712","2,216,467,626"


*Nota*  
El código anterrior devuelve:
- pelis_clean con los paises agrupados segun el diccionario
- pelis_countries con el id y los generos en formato dummies

## Agregar variable 'plataforma de straming'

La columna 'streaming' indica si la película fue estrenada en época de plataformas de streaming, considerando que el uso de plataformas de streaming para películas comenzó a popularizarse significativamente a partir de mediados de la década de 2010.  
Tomamos como año de referencia el 2012

In [254]:
pelis_clean['streaming'] = (pelis_clean['release_date'].dt.year >= 2012).astype(int)

## Reducir cantidad de idiomas

In [255]:
#Agrupo 'languages' en 'ingles' y 'otros'  
pelis_clean['language'] = pelis_clean['original_language'].apply(lambda x: 'ingles' if x == 'en' else 'otros') 

In [256]:
#elimino columna original
pelis_clean = pelis_clean.drop('original_language', axis=1)

## Renombrar columnas

In [257]:
pelis_clean.head(2)

,id,title,vote_average,vote_count,status,release_date,runtime,adult,genres,production_countries,year,decade,budget_adjust,revenue_adjust,streaming,language
0,27205,Inception,8,34495,Released,2010-07-15,148,False,Accion/Aventura,"Estados Unidos, Reino Unido",2010-01-01,2010,"248,677,072","1,283,069,190",0,ingles
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,"Accion/Aventura, Drama/Romance","Estados Unidos, Reino Unido",2014-01-01,2010,"226,089,773","961,538,161",1,ingles


In [258]:
pelis_clean.rename(columns={
    "id": "id",
    "title": "Título",
    "vote_average": "Promedio votos",
    "vote_count": "Conteo votos",
    "status": "Estado",
    "release_date": "Fecha estreno",
    "revenue": "Ingresos",
    "runtime": "Duración",
    "adult": "Adulto",
    "budget": "Presupuesto",
    "original_language": "Idioma original",
    "genres": "Géneros",
    "production_countries": "Países Producción",
    "year": "Año",
    "decade": "Década",
    "budget_adjust": "Presupuesto ajustado",
    "revenue_adjust": "Ingresos ajustados",
    "streaming": "Plataforma",
    "language": "Idioma"
}, inplace=True)

In [259]:
pelis_clean.head(2)

,id,Título,Promedio votos,Conteo votos,Estado,Fecha estreno,Duración,Adulto,Géneros,Países Producción,Año,Década,Presupuesto ajustado,Ingresos ajustados,Plataforma,Idioma
0,27205,Inception,8,34495,Released,2010-07-15,148,False,Accion/Aventura,"Estados Unidos, Reino Unido",2010-01-01,2010,"248,677,072","1,283,069,190",0,ingles
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,"Accion/Aventura, Drama/Romance","Estados Unidos, Reino Unido",2014-01-01,2010,"226,089,773","961,538,161",1,ingles


# Análisis: modelos de prediccion

## Generación DF para ML

In [260]:
# Eliminar las columnas no deseadas del DataFrame principal
pelis_ml = pelis_clean.drop(['Géneros', 'Países Producción', 'Década', 'Año', 'Estado', 'Título'], axis=1)

# Realizar la primera unión con pelis_genres
pelis_ml = pelis_ml.merge(
    pelis_genres,
    left_on=['id'],
    right_on=['id'],
    how='left'
)

# Realizar la segunda unión con pelis_countries
pelis_ml = pelis_ml.merge(
    pelis_countries,
    left_on=['id'],
    right_on=['id'],
    how='left'
)

#convierto booleanos a 0 y 1
pelis_ml = pelis_ml.applymap(lambda x: int(x) if isinstance(x, bool) else x)

#convierto columna idioma: ingles=1, otros=0
pelis_ml['Idioma'] = pelis_ml['Idioma'].replace({'ingles': 1, 'otros': 0})


C:\Users\Luis\AppData\Local\Temp\ipykernel_9304\3415690336.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pelis_ml = pelis_ml.applymap(lambda x: int(x) if isinstance(x, bool) else x)


In [261]:
pelis_ml.head()

,id,Promedio votos,Conteo votos,Fecha estreno,Duración,Adulto,Presupuesto ajustado,Ingresos ajustados,Plataforma,Idioma,...,España,Estados Unidos,Europe,Francia,India,Italia,Japon,Oceania,Reino Unido,Rusia
0,27205,8,34495,2010-07-15,148,0,"248,677,072","1,283,069,190",0,1,...,0,1,0,0,0,0,0,0,1,0
1,157336,8,32571,2014-11-05,169,0,"226,089,773","961,538,161",1,1,...,0,1,0,0,0,0,0,0,1,0
2,155,9,30619,2008-07-16,152,0,"306,229,401","1,662,839,626",0,1,...,0,1,0,0,0,0,0,0,1,0
3,19995,8,29815,2009-12-15,162,0,"380,140,206","4,689,528,313",0,1,...,0,1,0,0,0,0,0,0,1,0
4,24428,8,29166,2012-04-25,143,0,"321,054,712","2,216,467,626",1,1,...,0,1,0,0,0,0,0,0,0,0


In [262]:
mc = pelis_ml.corr()
mc

,id,Promedio votos,Conteo votos,Fecha estreno,Duración,Adulto,Presupuesto ajustado,Ingresos ajustados,Plataforma,Idioma,...,España,Estados Unidos,Europe,Francia,India,Italia,Japon,Oceania,Reino Unido,Rusia
id,1,0,-0,0,0,0,-0,-0,1,-0,...,0,-0,0,-0,0,-0,-0,0,-0,0
Promedio votos,0,1,0,-0,0,0,0,0,0,0,...,-0,0,-0,0,-0,0,0,0,0,-0
Conteo votos,-0,0,1,0,0,-0,1,1,0,0,...,-0,0,-0,-0,-0,-0,-0,0,0,-0
Fecha estreno,0,-0,0,1,0,0,0,-0,1,-0,...,0,-0,0,0,0,0,0,0,0,0
Duración,0,0,0,0,1,-0,0,0,0,-0,...,-0,-0,-0,-0,0,0,0,-0,0,-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Italia,-0,0,-0,0,0,0,-0,-0,-0,-0,...,0,-0,0,0,-0,1,0,-0,0,-0
Japon,-0,0,-0,0,0,0,0,-0,-0,-0,...,-0,-0,-0,0,-0,0,1,-0,0,-0
Oceania,0,0,0,0,-0,0,0,0,-0,0,...,-0,-0,-0,-0,-0,-0,-0,1,0,-0
Reino Unido,-0,0,0,0,0,-0,0,0,0,0,...,0,-0,0,0,-0,0,0,0,1,-0


In [263]:
mc['Promedio votos'].sort_values(ascending=False)

Promedio votos        1
Conteo votos          0
Duración              0
id                    0
Ingresos ajustados    0
                     ..
India                -0
Canada               -0
Terror/Misterio      -0
Todos los Publicos   -0
Rusia                -0
Name: Promedio votos, Length: 30, dtype: float64

In [264]:
mc['Ingresos ajustados'].sort_values(ascending=False)

Ingresos ajustados      1
Conteo votos            1
Presupuesto ajustado    1
Estados Unidos          0
Idioma                  0
                       ..
Francia                -0
Europe                 -0
India                  -0
id                     -0
Drama/Romance          -0
Name: Ingresos ajustados, Length: 30, dtype: float64

In [293]:
# genero variables para análisis
X = pelis_ml.drop(columns = ['id', 'Ingresos ajustados'])
y = pelis_ml['Ingresos ajustados']

#convierto la fecha a tipo numero
X['Fecha estreno'] = X['Fecha estreno'].astype('int64')

#separo datos en training y testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size = 0.75,
    test_size = 0.25,
    random_state=15
)

In [294]:
print(f" la dimension de X_train es {X_train.shape}")
print(f" la dimension de y_train es {y_train.shape}")
print(f" la dimension de X_test es {X_test.shape}")
print(f" la dimension de y_test es {y_test.shape}")

 la dimension de X_train es (7665, 28)
 la dimension de y_train es (7665,)
 la dimension de X_test es (2556, 28)
 la dimension de y_test es (2556,)


## Modelos de Regresion Lineal

### Estudio de multicolinealidad

#### Método del determinante

In [295]:
# calculo detemerinante de mc para los valores de training
np.linalg.det(X_train.corr()).round(4)

0.0088

*Nota*  
Resultado cercano a 0, indica correlación entre las variables

#### Cáclulo VIF

In [296]:
#convierto la fecha a tipo numero
#X_train['Fecha estreno'] = X_train['Fecha estreno'].astype('int64')
#X_test['Fecha estreno'] = X_test['Fecha estreno'].astype('int64')

#defino variables
var = X_train.columns.tolist()

#calculo VIF para cada una
vif_acum = {}

for i in var:

    x = X_train.loc[:,X_train.columns != i]
    y = X_train.loc[:,i]

    lm = LinearRegression()
    lm.fit(x, y)
    r2 = lm.score(x, y)

    vif = (1/(1-r2))
    vif_acum[i] = vif.round(2)

vif_acum

{'Promedio votos': 1.1,
 'Conteo votos': 1.43,
 'Fecha estreno': 1.69,
 'Duración': 1.05,
 'Adulto': 1.0,
 'Presupuesto ajustado': 1.4,
 'Plataforma': 1.57,
 'Idioma': 1.17,
 'Accion/Aventura': 1.09,
 'Documental/TV_Movie': 1.0,
 'Drama/Romance': 1.04,
 'Terror/Misterio': 1.0,
 'Todos los Publicos': 1.0,
 'Africa': 1.0,
 'Alemania': 1.01,
 'América': 1.01,
 'Asia': 1.02,
 'Canada': 1.0,
 'España': 1.01,
 'Estados Unidos': 1.2,
 'Europe': 1.02,
 'Francia': 1.01,
 'India': 1.05,
 'Italia': 1.0,
 'Japon': 1.0,
 'Oceania': 1.0,
 'Reino Unido': 1.01,
 'Rusia': 1.02}

*Nota*  
Todos los VIF son cercanos a 1, no se encuentra multicolinealidad en ninguna var en particular

### RFE

In [297]:
r2_acum = {}

for n in range(1, 29):
        
    estimator = LinearRegression()
    selector = RFE(estimator, n_features_to_select=n)
    
    # Ajustar el selector a los datos de entrenamiento
    selector = selector.fit(X_train, y_train)
    
    # Seleccionar variables relevantes
    X_train_final = X_train.loc[:, selector.support_]
    
    # Asegurarse de que X_test tenga las mismas columnas en el mismo orden que X_train_final
    X_test_final = X_test.loc[:, X_train_final.columns]
    
    lr = LinearRegression()
    lr.fit(X_train_final, y_train)
    r2 = lr.score(X_test_final, y_test).round(2)

    r2_acum[n] = r2

r2_acum

{1: 0.05,
 2: 0.08,
 3: 0.08,
 4: 0.08,
 5: 0.09,
 6: 0.09,
 7: 0.09,
 8: 0.12,
 9: 0.13,
 10: 0.12,
 11: 0.12,
 12: 0.12,
 13: 0.12,
 14: 0.12,
 15: 0.13,
 16: 0.13,
 17: 0.13,
 18: 0.13,
 19: 0.13,
 20: 0.13,
 21: 0.13,
 22: 0.13,
 23: 0.13,
 24: 0.13,
 25: 0.18,
 26: 0.5,
 27: 0.58,
 28: 0.58}

*Nota*  
Elijo modelo con 27 variables, R2 = 0,58.
Hay una la diferencia con 26 variables menos es significativa, sumar la 28 no.


### PCA

In [298]:
#estandarización de datos
media = X.mean()
desvio = X.std()

X_estandar = (X - media)/desvio
X_estandar

,Promedio votos,Conteo votos,Fecha estreno,Duración,Adulto,Presupuesto ajustado,Plataforma,Idioma,Accion/Aventura,Documental/TV_Movie,...,España,Estados Unidos,Europe,Francia,India,Italia,Japon,Oceania,Reino Unido,Rusia
0,2,11,0,2,-0,3,-1,1,1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,3,-0
1,2,11,1,3,-0,3,1,1,1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,3,-0
2,2,10,0,2,-0,4,-1,1,1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,3,-0
3,1,10,0,2,-0,6,-1,1,1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,3,-0
4,1,9,1,2,-0,5,1,1,1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,-0,-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10216,-0,0,1,-1,-0,-1,1,-2,-1,-0,...,-0,-2,-0,-0,-0,-0,-0,-0,-0,-0
10217,0,0,-0,-0,-0,-1,-1,1,-1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,-0,-0
10218,2,-0,1,-2,-0,-1,1,1,-1,-0,...,-0,1,-0,-0,-0,-0,-0,-0,-0,-0
10219,-0,0,1,1,-0,-1,1,-2,1,-0,...,-0,-2,-0,-0,4,-0,-0,-0,-0,-0


In [299]:
# aplicación de PCA 
pca = PCA(n_components=28)
X_PCA = pca.fit_transform(X_estandar)

# transformación de mtz original según las componentes hallaads
X_PCA = pd.DataFrame(X_PCA, columns=[f'PC{i}' for i in range(1, 29)])

In [300]:
#mtz de correlacion para las componentes trasnsformadas
X_PCA.corr().round(2)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28
PC1,1,0,0,0,-0,-0,-0,-0,-0,0,...,-0,-0,0,0,0,0,0,0,-0,-0
PC2,0,1,0,0,-0,-0,0,0,-0,0,...,0,0,0,-0,-0,0,-0,-0,0,-0
PC3,0,0,1,0,-0,0,0,0,-0,-0,...,0,-0,0,0,-0,0,0,0,-0,-0
PC4,0,0,0,1,0,-0,-0,-0,-0,0,...,-0,0,0,-0,-0,0,-0,0,-0,-0
PC5,-0,-0,-0,0,1,-0,0,-0,-0,0,...,0,-0,0,-0,0,0,0,-0,0,-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PC24,0,0,0,0,0,-0,-0,-0,0,-0,...,0,-0,-0,-0,0,1,-0,-0,0,0
PC25,0,-0,0,-0,0,0,0,0,0,-0,...,0,-0,-0,0,-0,-0,1,0,-0,-0
PC26,0,-0,0,0,-0,-0,0,0,0,-0,...,0,-0,0,0,-0,-0,0,1,-0,-0
PC27,-0,0,-0,-0,0,0,-0,0,0,-0,...,0,0,-0,0,0,0,-0,-0,1,-0


In [301]:
#variabilidad de las comoponentes halladas
variabilidad = pca.explained_variance_ratio_
variabilidad[0:15]

array([0.10524897, 0.07267294, 0.06077455, 0.05826463, 0.05155868,
       0.04620022, 0.04212127, 0.03992137, 0.03796192, 0.03717971,
       0.03667369, 0.03587934, 0.03512915, 0.03369362, 0.0333844 ])

In [302]:
#suma de la variabilidad para diferentes cantidades de componentes
var_acum = {}
for i in range(1,29):
    var_acum[i] = sum(variabilidad[0:i]).round(2)
var_acum

{1: 0.11,
 2: 0.18,
 3: 0.24,
 4: 0.3,
 5: 0.35,
 6: 0.39,
 7: 0.44,
 8: 0.48,
 9: 0.51,
 10: 0.55,
 11: 0.59,
 12: 0.62,
 13: 0.66,
 14: 0.69,
 15: 0.73,
 16: 0.76,
 17: 0.79,
 18: 0.82,
 19: 0.85,
 20: 0.88,
 21: 0.9,
 22: 0.93,
 23: 0.94,
 24: 0.96,
 25: 0.97,
 26: 0.98,
 27: 1.0,
 28: 1.0}

*Nota*  
Con 19 de las 28 variables se explican el 85% de los resultados

In [303]:
r2_acum = {}

for i in range(1,29):
    X_final = X_PCA.iloc[:,0:i]
    
    X_train_final, X_test_final= train_test_split(
        X_final,
        train_size = 0.75,
        test_size = 0.25,
        random_state=15
)

    lr = LinearRegression()
    lr.fit(X_train_final, y_train)
    r2_acum[i] = lr.score(X_test_final, y_test).round(2)

r2_acum

{1: 0.15,
 2: 0.38,
 3: 0.39,
 4: 0.4,
 5: 0.42,
 6: 0.47,
 7: 0.47,
 8: 0.48,
 9: 0.48,
 10: 0.48,
 11: 0.49,
 12: 0.49,
 13: 0.49,
 14: 0.49,
 15: 0.5,
 16: 0.5,
 17: 0.51,
 18: 0.51,
 19: 0.51,
 20: 0.51,
 21: 0.51,
 22: 0.55,
 23: 0.58,
 24: 0.58,
 25: 0.58,
 26: 0.59,
 27: 0.59,
 28: 0.59}

*Nota*  
Con 23 variables el r2 es 0,58, de ahi en más el modelo no mejora significativamente.

## Arboles y Random Forest

### Modelo de árbol

In [304]:
X_train.head()

,Promedio votos,Conteo votos,Fecha estreno,Duración,Adulto,Presupuesto ajustado,Plataforma,Idioma,Accion/Aventura,Documental/TV_Movie,...,España,Estados Unidos,Europe,Francia,India,Italia,Japon,Oceania,Reino Unido,Rusia
3698,8,833,1341014400000000000,85,0,"1,459,340",1,1,0,1,...,0,1,1,0,0,0,0,0,1,0
2376,6,1653,1677715200000000000,92,0,"46,440,000",1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2707,6,1396,1141344000000000000,104,0,"21,155,135",0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
3036,7,1190,982281600000000000,119,0,"82,545,429",0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5854,5,249,1034294400000000000,89,0,"19,996,470",0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


In [305]:
r2_acum= {}

for n in range (3,16):
    # Crear el modelo de árbol de decisión
    clf = DecisionTreeRegressor(max_depth=n, random_state=15)
    
    # Entrenar el modelo
    clf.fit(X_train, y_train)
    
    # Realizar predicciones
    y_pred = clf.predict(X_test)
    
    # Evaluar el modelo con métricas de regresión
    r2_acum [n] = r2_score(y_test, y_pred).round(2)

r2_acum

{3: 0.48,
 4: 0.49,
 5: 0.44,
 6: 0.53,
 7: 0.48,
 8: 0.39,
 9: 0.37,
 10: 0.54,
 11: 0.48,
 12: 0.39,
 13: 0.41,
 14: 0.45,
 15: 0.44}

###### *Nota*  
Indicando una profundidad de 6, se obtiene un r2 de 0,53

### Modelo de bosque

In [306]:
r2_acum = {}

for n in range (3,26):
    # Crear el modelo de bosque
    rf = RandomForestRegressor(max_depth=n, random_state=15)
    
    # Entrenar el modelo
    rf.fit(X_train, y_train)
    
    # Realizar predicciones
    y_pred = rf.predict(X_test)
    
    # Evaluar el modelo con métricas de regresión
    r2_acum [n] =r2_score(y_test, y_pred).round(2)

r2_acum

{3: 0.52,
 4: 0.57,
 5: 0.59,
 6: 0.61,
 7: 0.61,
 8: 0.62,
 9: 0.62,
 10: 0.62,
 11: 0.63,
 12: 0.62,
 13: 0.63,
 14: 0.62,
 15: 0.62,
 16: 0.63,
 17: 0.63,
 18: 0.63,
 19: 0.63,
 20: 0.63,
 21: 0.62,
 22: 0.62,
 23: 0.62,
 24: 0.62,
 25: 0.62}

*Nota*  
Con una profundidad de 11 y 100 árboles, se obtiene un R2 de 0,63, más allá de eso el modelo no mejora significativamente

## Evaluación de modelos

In [370]:
pelis_clean.head(2)

,id,Título,Promedio votos,Conteo votos,Estado,Fecha estreno,Duración,Adulto,Géneros,Países Producción,Año,Década,Presupuesto ajustado,Ingresos ajustados,Plataforma,Idioma
0,27205,Inception,8,34495,Released,2010-07-15,148,False,Accion/Aventura,"Estados Unidos, Reino Unido",2010-01-01,2010,"248,677,072","1,283,069,190",0,ingles
1,157336,Interstellar,8,32571,Released,2014-11-05,169,False,"Accion/Aventura, Drama/Romance","Estados Unidos, Reino Unido",2014-01-01,2010,"226,089,773","961,538,161",1,ingles


In [371]:
#genero una lista de peelis protegonizadas por Leo
pelis_Leo = ['Shutter Island', 'Inception', 'The Beach', 'Gangs of New York', 'The Departed'] 
pelis_Leo

['Shutter Island',
 'Inception',
 'The Beach',
 'Gangs of New York',
 'The Departed']

In [372]:
#genero lista de índices de esas películas en el dataset original
indices = pelis_clean[pelis_clean['Título'].isin(pelis_Leo)].index

In [373]:
# genero un diccionaro con el índice, el título y el ingreso ajsutado de cada peli de acuerdo al dataset original
eval = {'indice': [], 'Título': [], 'Ingresos ajustados': []}

# Iterar sobre los índices
for i in indices:
    tit = pelis_clean.loc[i, 'Título']
    ingr = pelis_clean.loc[i, 'Ingresos ajustados']
    
    # Almacenar los valores en las listas correspondientes
    eval['indice'].append(i)
    eval['Título'].append(tit)
    eval['Ingresos ajustados'].append(ingr)

eval


{'indice': [0, 21, 136, 630, 1059],
 'Título': ['Inception',
  'Shutter Island',
  'The Departed',
  'Gangs of New York',
  'The Beach'],
 'Ingresos ajustados': [1283069189.787076,
  458187504.6563628,
  513831780.727467,
  387476610.3064644,
  306793898.7057282]}

### RFE

In [374]:
#crear el modelo de regresion lineal con 27 variables seleccionadas por RFE
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=27)
    
# Ajustar el selector a los datos de entrenamiento
selector = selector.fit(X_train, y_train)
    
# Seleccionar variables relevantes
X_train_final = X_train.loc[:, selector.support_]
    
# Asegurarse de que X_test tenga las mismas columnas en el mismo orden que X_train_final
X_test_final = X_test.loc[:, X_train_final.columns]

#generar modelo, entrenar y evaluar
lr_RFE = LinearRegression()
lr_RFE.fit(X_train_final, y_train)
lr_RFE.score(X_test_final, y_test).round(2)

0.58

In [375]:
#ajusto el dataset de variable a evaluar con las 27 variables seleccionadas por RFE 
X_RFE = X.loc[:, selector.support_]

In [376]:
#agrego al diccionario las predicciones del modelo RFE
eval['Ingresos RFE'] = []
for i in indices:
    elemento = X_RFE.iloc[i].values.reshape(1, -1)
    ingr = lr_RFE.predict(elemento)
    eval['Ingresos RFE'].append(ingr[0]) 

eval

C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


{'indice': [0, 21, 136, 630, 1059],
 'Título': ['Inception',
  'Shutter Island',
  'The Departed',
  'Gangs of New York',
  'The Beach'],
 'Ingresos ajustados': [1283069189.787076,
  458187504.6563628,
  513831780.727467,
  387476610.3064644,
  306793898.7057282],
 'Ingresos RFE': [1956277843.005501,
  1191696437.9230742,
  865923444.3758487,
  225394979.38945857,
  688346240.0798832]}

### PCA

In [377]:
#crear el modelo de regresion lineal con las 23 componentes principales
X_PCA = X_PCA.iloc[:,0:23]
    
X_train_final, X_test_final= train_test_split(
        X_PCA,
        train_size = 0.75,
        test_size = 0.25,
        random_state=15
)

lr_PCA = LinearRegression()
lr_PCA.fit(X_train_final, y_train)
lr_PCA.score(X_test_final, y_test).round(2)

0.58

In [378]:
#agrego al diccionario las predicciones del modelo PCA
eval['Ingresos PCA'] = []
for i in indices:
    elemento = X_PCA.iloc[i].values.reshape(1, -1)
    ingr = lr_PCA.predict(elemento)
    eval['Ingresos PCA'].append(ingr[0]) 

eval

C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


{'indice': [0, 21, 136, 630, 1059],
 'Título': ['Inception',
  'Shutter Island',
  'The Departed',
  'Gangs of New York',
  'The Beach'],
 'Ingresos ajustados': [1283069189.787076,
  458187504.6563628,
  513831780.727467,
  387476610.3064644,
  306793898.7057282],
 'Ingresos RFE': [1956277843.005501,
  1191696437.9230742,
  865923444.3758487,
  225394979.38945857,
  688346240.0798832],
 'Ingresos PCA': [1965795555.2123146,
  1179300590.6224027,
  875158220.744502,
  217808593.0382157,
  680838303.4070767]}

### Modelo de árbol

In [379]:
# Crear el modelo de árbol de decisión con 6 niveles de profundidad
clf = DecisionTreeRegressor(max_depth=6, random_state=15)
    
# Entrenar el modelo
clf.fit(X_train, y_train)
    
# Realizar predicciones
y_pred = clf.predict(X_test)
    
# Evaluar el modelo con métricas de regresión
r2_score(y_test, y_pred).round(2)

0.53

In [380]:
#agrego al diccionario las predicciones del modelo de árbol
eval['Ingresos ARBOL'] = []
for i in indices:
    elemento = X.iloc[i].values.reshape(1, -1)
    ingr = clf.predict(elemento)
    eval['Ingresos ARBOL'].append(ingr[0]) 

eval

C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


{'indice': [0, 21, 136, 630, 1059],
 'Título': ['Inception',
  'Shutter Island',
  'The Departed',
  'Gangs of New York',
  'The Beach'],
 'Ingresos ajustados': [1283069189.787076,
  458187504.6563628,
  513831780.727467,
  387476610.3064644,
  306793898.7057282],
 'Ingresos RFE': [1956277843.005501,
  1191696437.9230742,
  865923444.3758487,
  225394979.38945857,
  688346240.0798832],
 'Ingresos PCA': [1965795555.2123146,
  1179300590.6224027,
  875158220.744502,
  217808593.0382157,
  680838303.4070767],
 'Ingresos ARBOL': [1293448370.7368631,
  712163545.4594067,
  1293448370.7368631,
  125050489.10651103,
  459517423.0764366]}

### Modelo de bosque

In [381]:
# Crear el modelo de bosque con 100 árboles y 6 niveles de profundidad
rf = RandomForestRegressor(n_estimators=100, max_depth=11, random_state=15)

# Entrenar el modelo
rf.fit(X_train, y_train)

# Realizar predicciones
y_pred = rf.predict(X_test)

# Evaluar el modelo
r2_score(y_test, y_pred).round(2)

0.63

In [382]:
#agrego al diccionario las predicciones del modelo de bosque
eval['Ingresos BOSQUE'] = []
for i in indices:
    elemento = X.iloc[i].values.reshape(1, -1)
    ingr = rf.predict(elemento)
    eval['Ingresos BOSQUE'].append(ingr[0]) 

eval

C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Luis\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


{'indice': [0, 21, 136, 630, 1059],
 'Título': ['Inception',
  'Shutter Island',
  'The Departed',
  'Gangs of New York',
  'The Beach'],
 'Ingresos ajustados': [1283069189.787076,
  458187504.6563628,
  513831780.727467,
  387476610.3064644,
  306793898.7057282],
 'Ingresos RFE': [1956277843.005501,
  1191696437.9230742,
  865923444.3758487,
  225394979.38945857,
  688346240.0798832],
 'Ingresos PCA': [1965795555.2123146,
  1179300590.6224027,
  875158220.744502,
  217808593.0382157,
  680838303.4070767],
 'Ingresos ARBOL': [1293448370.7368631,
  712163545.4594067,
  1293448370.7368631,
  125050489.10651103,
  459517423.0764366],
 'Ingresos BOSQUE': [1639636543.3409061,
  592639261.5088515,
  655852652.3454643,
  149366576.14528787,
  546805269.0876052]}

### EVALUACION

In [21]:
comp = {
    'modelos': ['RegLineal RFE', 'RegLineal PCA', 'ÁRBOL', 'BOSQUE'],
    'R2': [0.58, 0.58, 0.53, 0.63 ]
}

compR2 = pd.DataFrame(comp)

# Transponer el DataFrame y configurar la primera fila como las nuevas columnas
compR2_transposed = compR2.set_index('modelos').T

# Renombrar las columnas para que sean los valores originales en lugar de números
compR2_transposed.columns.name = None

# Mostrar el DataFrame modificado
compR2_transposed

,RegLineal RFE,RegLineal PCA,ÁRBOL,BOSQUE
R2,0.58,0.58,0.53,0.63


In [383]:
# Crear el DataFrame a partir del diccionario
comparativa = pd.DataFrame(eval)

# Configurar la columna 'titulo' y eliminar columna 'indice'como índice del DataFrame
comparativa.set_index('Título', inplace=True)
comparativa.drop(columns='indice', inplace=True)

comparativa

,Ingresos ajustados,Ingresos RFE,Ingresos PCA,Ingresos ARBOL,Ingresos BOSQUE
Título,,,,,
Inception,"1,283,069,190","1,956,277,843","1,965,795,555","1,293,448,371","1,639,636,543"
Shutter Island,"458,187,505","1,191,696,438","1,179,300,591","712,163,545","592,639,262"
The Departed,"513,831,781","865,923,444","875,158,221","1,293,448,371","655,852,652"
Gangs of New York,"387,476,610","225,394,979","217,808,593","125,050,489","149,366,576"
The Beach,"306,793,899","688,346,240","680,838,303","459,517,423","546,805,269"
